<a href="https://colab.research.google.com/github/hianaracamilla/inadimplente-wlc/blob/main/Inadimplentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FILTRANDO INADIMPLENTES**

---



Importando Pandas

In [4]:
import pandas as pd

Importar planilha google que contem os logins

In [5]:
sheet_id = '1e2RG1Bbb4bfGwM0eYMzkUJqLL_jckorMTFGXJ6BdpIo'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet'
df_logins = pd.read_csv(url)

Importar base de viagens e verifica quantas tabelas possui

In [6]:
inadimplente_html = pd.read_html('/content/inadimplentes_atual.html')
len(inadimplente_html)

6

Selecionar tabela que estão as viagens

In [7]:
df = inadimplente_html[3]
df.head(6)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,Viajes por Vendedor [ver sin detalle de pagos...,Viajes por Vendedor [ver sin detalle de pagos...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Filtrar por Vendedor: [admin] Amanda Casale ...,Filtrar por Vendedor: [admin] Amanda Casale ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Filtrar por Vendedor:,[admin] Amanda Casale [admin] Joaquim Adm [a...,Filtrar por Operador:,Todos [conductor] Benjamin Layana [conductor...,TOUR:,Seleccione Tour Acordo Gereencia Acordo Gerê...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Desde:,NaN,Hasta:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Intervalo de tempo: de lunes 01-07-2024 a mart...,Intervalo de tempo: de lunes 01-07-2024 a mart...,Intervalo de tempo: de lunes 01-07-2024 a mart...,Intervalo de tempo: de lunes 01-07-2024 a mart...,Intervalo de tempo: de lunes 01-07-2024 a mart...,Intervalo de tempo: de lunes 01-07-2024 a mart...,Intervalo de tempo: de lunes 01-07-2024 a mart...,Intervalo de tempo: de lunes 01-07-2024 a mart...,Intervalo de tempo: de lunes 01-07-2024 a mart...,Intervalo de tempo: de lunes 01-07-2024 a mart...,Intervalo de tempo: de lunes 01-07-2024 a mart...,Intervalo de tempo: de lunes 01-07-2024 a mart...,Intervalo de tempo: de lunes 01-07-2024 a mart...,Intervalo de tempo: de lunes 01-07-2024 a mart...,Intervalo de tempo: de lunes 01-07-2024 a mart...,Intervalo de tempo: de lunes 01-07-2024 a mart...,Intervalo de tempo: de lunes 01-07-2024 a mart...,Intervalo de tempo: de lunes 01-07-2024 a mart...,Intervalo de tempo: de lunes 01-07-2024 a mart...
5,Viajes Realizados,Viajes Realizados,Viajes Realizados,Viajes Realizados,Viajes Realizados,Viajes Realizados,Viajes Realizados,Viajes Realizados,Viajes Realizados,Viajes Realizados,Viajes Realizados,Viajes Realizados,Viajes Realizados,Viajes Realizados,Viajes Realizados,Viajes Realizados,Viajes Realizados,Viajes Realizados,Viajes Realizados


Apagar linhas e colunas que não serão utilizadas

In [8]:
df.drop([0,1,2,3,4,5], axis=0, inplace=True)
df.drop([1, 6, 7, 8, 9, 11, 12, 14, 15, 16, 17, 18], axis=1, inplace=True)

Remover telefones duplicados e trata data deixando somente numeros

In [9]:
df = df.drop_duplicates([10])
df[0] = df[0].str.split(' ').str[1]

<ipython-input-9-8a26bfb9785c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[0] = df[0].str.split(' ').str[1]


Atribuir o nome data e Monto_Pendiente a listas de cabeçalho

In [10]:
df.iloc[0, 0] = 'Data'
df.iloc[0, 6] = 'Monto_Pendiente'

Promover primeira linha para cabeçalho

In [11]:
df.columns = df.iloc[0]
df = df[1:]

Função para limpar extrarir números de textos

In [12]:
def convert_to_int(value):
    try:
        value = str(value)
        value = value.replace('$', '').replace(' ', '').replace('.', '')
        return int(value)
    except (ValueError, AttributeError):
        return None

[texto del vínculo](https://)Aplicando função na coluna Monto_Pendiente

In [13]:
df['Monto_Pendiente'] = df['Monto_Pendiente'].apply(convert_to_int)

Criando filtro em monto_pendiente onde o valor em débito é maior que 1000 para considerar inadimplente e soma para saber quantos clientes estão inadimplentes

In [14]:
selecao_inadimplente = df['Monto_Pendiente'] > 999
selecao_inadimplente.sum()

171

Criando DF Inadimplente com o filtro

In [15]:
inadimplente = df[selecao_inadimplente]

Criando Merge para saber qual setor de cada Login

In [16]:
inadimplente = inadimplente.merge(df_logins, how='left', left_on='Vendedor', right_on='LOGIN')

Apagando linhas que não tem setor, pois são transpaso

In [17]:
inadimplente = inadimplente.dropna(subset=['SETOR'])
inadimplente.drop('LOGIN', axis=1, inplace=True)

Verifica qual é a data máxima para confirmar se é igual a hoje

In [18]:
inadimplente['Data'].max()

'30-07'

Altera a ordem das colunas

In [19]:
new_order = ['ID', 'Destino', 'Vendedor', 'SETOR', 'Data', 'Nome', 'Telefone', 'Monto_Pendiente']
inadimplente = inadimplente[new_order]

Exportando tudo para excel

In [20]:
inadimplente.to_excel('inadimplentes_atual.xlsx', index=False)